# Analyze aggregated household profiles

In this notebook, the resulting profiles from the household-level simulation are analyzed and compared. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# LOADING PRICE DATA

df_ev = pd.read_pickle("./input/preprocessed/2019 Hamelin 500 EV.pkl")
df_price = pd.read_excel("./input/Gro_handelspreise_201901010000_201912312359_Stunde.xlsx", skiprows=9)
df_p = pd.DataFrame(index=df_ev.index)
# MWh prices are transformed to kWh prices
df_p["Deutschland/Luxemburg [€/kWh]"] = df_price["Deutschland/Luxemburg [€/MWh]"].apply(lambda x: x/1000).values
min_price = df_p["Deutschland/Luxemburg [€/kWh]"].min()
df_p["Deutschland/Luxemburg [€/kWh]"] = df_p["Deutschland/Luxemburg [€/kWh]"].apply(lambda x: x+abs(min_price)) # avoid negative values for optimization


In [ ]:
# FUNCTIONS

def calculate_peaks_based_on_case(target_df, case):
    target_df["Sum"] = target_df.sum(axis=1)
    target_df = target_df[["Sum"]]
    heat_pump_peak_day = '2019-12-01'
    pv_peak_day = '2019-05-13'
    pv_peak = target_df["Sum"][pv_peak_day].abs().max()
    heat_pump_peak_day = '2019-12-01'
    hp_peak = target_df["Sum"][heat_pump_peak_day].abs().max()
    ev_peak_day = "2019-11-18"
    ev_peak = target_df["Sum"][ev_peak_day].abs().max()
    total_peak = target_df["Sum"].abs().max()
    
    case["worst_hp_day_peak_load"] = hp_peak
    case["worst_pv_day_peak_load"] = pv_peak
    case["worst_ev_day_peak_load"] = ev_peak
    case["worst_total_peak_load"] = total_peak
    
def calculate_agg_hp_consumption(target_df, case):
    target_df["Sum"] = target_df.sum(axis=1)
    total_hp_consumption = target_df["Sum"].sum()
    case["yearly_hp_consumption"] = total_hp_consumption
    
def calculate_agg_ev_consumption(target_df, case):
    target_df["Sum"] = target_df.sum(axis=1)
    total_ev_consumption = target_df["Sum"].sum()
    case["yearly_ev_consumption"] = total_ev_consumption

def settings_to_target_df(pricing_type, operation_type, feed_in_type,grid_charge_type, grid_charging_allowed,hp=False,ev=False,costs=False):
    target_df = pd.read_pickle(f"./output/00_pricing_{pricing_type}_operation_{operation_type}_fi_{feed_in_type}_ne_{str(grid_charge_type)}_gridch_{str(grid_charging_allowed)}.pkl")
    if hp:
        target_df = pd.read_pickle(f"./output/detailed_consumption/HP_00_pricing_{pricing_type}_operation_{operation_type}_fi_{feed_in_type}_ne_{str(grid_charge_type)}_gridch_{str(grid_charging_allowed)}new.pkl")
    elif ev:
        target_df = pd.read_pickle(f"./output/detailed_consumption/EV_00_pricing_{pricing_type}_operation_{operation_type}_fi_{feed_in_type}_ne_{str(grid_charge_type)}_gridch_{str(grid_charging_allowed)}new.pkl")
    
    elif costs:
        target_df = pd.read_pickle(f"./output/detailed_financials/Costs_00_pricing_{pricing_type}_operation_{operation_type}_fi_{feed_in_type}_ne_{str(grid_charge_type)}_gridch_{str(grid_charging_allowed)}new.pkl")

    return target_df



In [ ]:
# DEFINING REGULATORY CASES WITH OR WITHOUT DYNAMIC TARIFF ADOPTION

case_constant_without_flexibility = {"name":"Constant Volumetric_FIT without flexiblity","feed_in_tariff" :"fit", "grid_charge_type" : "volumetric", "pricing_type" : "constant", "operation_type" : "constant","grid_charging_allowed":"False"}

case_volumetric_fit_gcfalse_constant = {"name":"Constant Volumetric_FIT","feed_in_tariff" :"fit", "grid_charge_type" : "volumetric", "pricing_type" : "constant", "operation_type" : "dynamic","grid_charging_allowed":"False"}
case_volumetric_fit_gcfalse = {"name":"Dynamic Volumetric_FIT","feed_in_tariff" :"fit", "grid_charge_type" : "volumetric", "pricing_type" : "dynamic", "operation_type" : "dynamic","grid_charging_allowed":"False"}

case_volumetric_dynamic_gcfalse_constant = {"name":"Constant Volumetric_DynFeed","feed_in_tariff" :"dynamic", "grid_charge_type" : "volumetric", "pricing_type" : "constant", "operation_type" : "dynamic","grid_charging_allowed":"False"}
case_volumetric_dynamic_gcfalse = {"name":"Dynamic Volumetric_DynFeed","feed_in_tariff" :"dynamic", "grid_charge_type" : "volumetric", "pricing_type" : "dynamic", "operation_type" : "dynamic","grid_charging_allowed":"False"}

case_peak_fit_gcfalse_constant = {"name":"Constant Peak_FIT","feed_in_tariff" :"fit", "grid_charge_type" : "peak", "pricing_type" : "constant", "operation_type" : "dynamic","grid_charging_allowed":"False"}
case_peak_fit_gcfalse = {"name":"Dynamic Peak_FIT","feed_in_tariff" :"fit", "grid_charge_type" : "peak", "pricing_type" : "dynamic", "operation_type" : "dynamic","grid_charging_allowed":"False"}

case_peak_dynamic_gcfalse_constant = {"name":"Constant Peak_DynFeed","feed_in_tariff" :"dynamic", "grid_charge_type" : "peak", "pricing_type" : "constant", "operation_type" : "dynamic","grid_charging_allowed":"False"}
case_peak_dynamic_gcfalse = {"name":"Dynamic Peak_DynFeed","feed_in_tariff" :"dynamic", "grid_charge_type" : "peak", "pricing_type" : "dynamic", "operation_type" : "dynamic","grid_charging_allowed":"False"}

case_rotating_fit_gcfalse_constant = {"name":"Constant Rotating_FIT","feed_in_tariff" :"fit", "grid_charge_type" : "rotating", "pricing_type" : "constant", "operation_type" : "dynamic","grid_charging_allowed":"False"}
case_rotating_fit_gcfalse = {"name":"Dynamic Rotating_FIT","feed_in_tariff" :"fit", "grid_charge_type" : "rotating", "pricing_type" : "dynamic", "operation_type" : "dynamic","grid_charging_allowed":"False"}

case_rotating_dynamic_gcfalse_constant = {"name":"Constant Rotating_DynFeed","feed_in_tariff" :"dynamic", "grid_charge_type" : "rotating", "pricing_type" : "constant", "operation_type" : "dynamic","grid_charging_allowed":"False"}
case_rotating_dynamic_gcfalse = {"name":"Dynamic Rotating_DynFeed","feed_in_tariff" :"dynamic", "grid_charge_type" : "rotating", "pricing_type" : "dynamic", "operation_type" : "dynamic","grid_charging_allowed":"False"}

case_segmented_fit_gcfalse_constant = {"name":"Constant Segmented_FIT","feed_in_tariff" :"fit", "grid_charge_type" : "segmented", "pricing_type" : "constant", "operation_type" : "dynamic","grid_charging_allowed":"False"}
case_segmented_fit_gcfalse = {"name":"Dynamic Segmented_FIT","feed_in_tariff" :"fit", "grid_charge_type" : "segmented", "pricing_type" : "dynamic", "operation_type" : "dynamic","grid_charging_allowed":"False"}

case_segmented_dynamic_gcfalse_constant = {"name":"Constant Segmented_DynFeed","feed_in_tariff" :"dynamic", "grid_charge_type" : "segmented", "pricing_type" : "constant", "operation_type" : "dynamic","grid_charging_allowed":"False"}
case_segmented_dynamic_gcfalse = {"name":"Dynamic Segmented_DynFeed","feed_in_tariff" :"dynamic", "grid_charge_type" : "segmented", "pricing_type" : "dynamic", "operation_type" : "dynamic","grid_charging_allowed":"False"}




cases = [
    case_constant_without_flexibility,
    case_volumetric_fit_gcfalse_constant,
    case_volumetric_fit_gcfalse,
    case_volumetric_dynamic_gcfalse_constant,
    case_volumetric_dynamic_gcfalse,
    case_peak_fit_gcfalse_constant,
    case_peak_fit_gcfalse,
    case_peak_dynamic_gcfalse_constant,
    case_peak_dynamic_gcfalse,
    case_rotating_fit_gcfalse_constant,
    case_rotating_fit_gcfalse,
    case_rotating_dynamic_gcfalse_constant,
    case_rotating_dynamic_gcfalse,
    case_segmented_fit_gcfalse_constant,
    case_segmented_fit_gcfalse,
    case_segmented_dynamic_gcfalse_constant,
    case_segmented_dynamic_gcfalse
]


for case in cases: 
    target_df_consumption = settings_to_target_df(case["pricing_type"], case["operation_type"], case["feed_in_tariff"],case["grid_charge_type"],
                                                     case["grid_charging_allowed"])
    
    calculate_peaks_based_on_case(target_df_consumption, case)


In [ ]:
# SAVE RESULTING PEAK LOADS TO LATEX IN OUTPUT FOLDER

paper_case_overview = cases_comparison.copy()
paper_case_overview = paper_case_overview[["feed_in_tariff","grid_charge_type","pricing_type","operation_type"]]
paper_case_overview = paper_case_overview.rename({"feed_in_tariff":"FIT","grid_charge_type":"Grid Charges","pricing_type":"Electricity Tariff","operation_type":"Operation" 
                           },axis=1)
paper_case_overview.to_latex("./output/case_overview.tex")

paper_case_results = cases_comparison.drop(["feed_in_tariff","grid_charge_type","pricing_type","operation_type","grid_charging_allowed"],axis=1).round(1)
paper_case_results = paper_case_results.rename({"worst_hp_day_peak_load":"Heat pump peak [kW]","worst_pv_day_peak_load":"Feed-in peak [kW]","worst_ev_day_peak_load":"EV peak [kW]","worst_total_peak_load":"Yearly peak [kW]"},axis=1)
paper_case_results.to_latex("./output/case_peaks.tex")


In [ ]:
# VISUALIZE TABLE IN NOTEBOOk
cases_comparison = pd.DataFrame(cases).set_index("name")

cases_comparison[["worst_hp_day_peak_load","worst_pv_day_peak_load","worst_ev_day_peak_load","worst_total_peak_load"]].style.background_gradient(cmap='RdYlGn_r')